In [1]:
# pip install qdrant-client sentence-transformers llama-index

In [2]:
import json
with open("stoic_texts.json", "r", encoding="utf-8") as f:
    data = json.load(f)
texts = data["texts"]
print(f"Total texts extracted: {len(texts)}")
print("Sample texts:", texts[:3])

Total texts extracted: 1606
Sample texts: ['From my grandfather Verus I learned good morals and the government of my temper.', 'From the reputation and remembrance of my father, modesty and a manly character.', 'From Fronto I learned to observe what envy, and duplicity, and hypocrisy are in a tyrant, and that\ngenerally those among us who are called Patricians are rather deficient in paternal affection.']


In [3]:
# pip install llama-index
# pip install llama-index-vector-stores-qdrant llama-index-embeddings-fastembed
# pip install llama-index-readers-file
# pip install llama-index-llms-groq  # deepseek r1

In [4]:
import logging
import sys
import os

In [5]:
from llama_index.core import Document

In [6]:
import qdrant_client
from qdrant_client import models
from llama_index.core import Document  # Added this
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.groq import Groq  # deep seek r1 implementation

c:\Users\LENOVO\Desktop\Stoic gpt\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
documents = [Document(text=text) for text in texts]
print(f"Created {len(documents)} documents")

Created 1606 documents


In [8]:
len(documents)

1606

In [9]:
documents[100].text

"Therefore you should state why the wise man ought not to get drunk. Explain by facts, and not by\nmere words, the hideousness of the thing, and its haunting evils. Do that which is easiest of all ?\nnamely, demonstrate that what men call pleasures are punishments as soon as they have exceeded\ndue bounds. For if you try to prove that the wise man can souse himself with much wine and yet\nkeep his course straight, even though he be in his cups, you may go on to infer by syllogisms that\nhe will not die if he swallows poison, that he will not sleep if he takes a sleeping-potion, that he will\nnot vomit and reject the matter which clogs his stomach when you give him hellebore. But, when a\nman's feet totter and his tongue is unsteady, what reason have you for believing that he is half sober\nand half drunk? Farewell."

In [10]:
# from google.colab import userdata

In [11]:
from dotenv import load_dotenv
import os

# Specify your env file name
load_dotenv("keys.env")  # loads environment variables from keys.env

# Now you can access your secrets like this:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

In [12]:
# os.environ['GROQ_API_KEY'] = userdata.get("GROQ_API_KEY")

In [13]:
# llm = Groq(model="deepseek-r1-distill-llama-70b")

In [14]:
# llm.complete("How You Doing").text

In [17]:
from llama_index.llms.groq import Groq

llm = Groq(
    api_key=GROQ_API_KEY,  # Make sure GROQ_API_KEY is loaded from your .env
    model="deepseek-r1-distill-llama-70b",
    default_headers={},    
    logprobs=None
)

print(llm.complete("How You Doing").text)

<think>

</think>

Doing well, thanks for asking! How can I assist you today? 😊


In [18]:
embed_model = FastEmbedEmbedding(model_name="thenlper/gte-large")

c:\Users\LENOVO\Desktop\Stoic gpt\venv\Lib\site-packages\llama_index\embeddings\fastembed\base.py:88: UserWarning: The model thenlper/gte-large now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  self._model = TextEmbedding(


In [19]:
client = qdrant_client.QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True
)

In [20]:
collection_name = "stoic_corpus"

In [21]:
if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=1024,
                                           distance=models.Distance.COSINE,on_disk=True),
        quantization_config=models.BinaryQuantization(
            binary=models.BinaryQuantizationConfig(
                always_ram=True,
            ),
        ),
    )
else:
    print("Collection already exists")

Collection already exists


In [ ]:
embeddings = []
BATCH_SIZE = 50

In [ ]:
for page in range(0, len(texts), BATCH_SIZE):
    page_content = texts[page:page + BATCH_SIZE]
    response = embed_model.get_text_embedding_batch(page_content)
    embeddings.extend(response)

In [ ]:
BATCH_SIZE = 10  # Try 10 or even 5 if you still get errors

for idx in range(0, len(texts), BATCH_SIZE):
    docs = texts[idx:idx + BATCH_SIZE]
    embeds = embeddings[idx:idx + BATCH_SIZE]

    client.upload_collection(
        collection_name=collection_name,
        vectors=embeds,
        payload=[{"context": doc} for doc in docs]  # Use doc, not context
    )

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_7560\2319677760.py:7: UserWarning: Batch upload failed 1 times. Retrying...
  client.upload_collection(
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_7560\2319677760.py:7: UserWarning: Batch upload failed 2 times. Retrying...
  client.upload_collection(


In [ ]:
client.update_collection(collection_name= collection_name,
                        optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000))

True

## RETRIEVAL AUGMENTED GENERATION


In [ ]:
def search(query,k=5):
  # query = user prompt
  query_embedding = embed_model.get_query_embedding(query)
  result = client.query_points(
            collection_name = collection_name,
            query=query_embedding,
            limit = k
        )
  return result

In [ ]:
relevant_docs = search("How can I stay calm in tough times?")

In [ ]:
len(relevant_docs.points)

5

In [ ]:
print(relevant_docs.points[4].payload['context'])

You write me that you are anxious about the result of a lawsuit, with which an angry opponent is
threatening you; and you expect me to advise you to picture to yourself a happier issue, and to rest
in the allurements of hope. Why, indeed, is it necessary to summon trouble, ? which must be
endured soon enough when it has once arrived, ? or to anticipate trouble and ruin the present
through fear of the future? It is indeed foolish to be unhappy now because you may be unhappy at
some future time. But I shall conduct you to peace of mind by another route: if you would put off all
worry, assume that what you fear may happen will certainly happen in any event; whatever the
trouble may be, measure it in your own mind, and estimate the amount of your fear. You will thus
understand that what you fear is either insignificant or short-lived. And you need not spend a long
time in gathering illustrations which will strengthen you; every epoch has produced them. Let your
thoughts travel into any era

In [ ]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

message_templates = [
    ChatMessage(
        content="""
        You are an expert Stoic assistant, well-versed in the teachings of Stoicism, including the works of Marcus Aurelius, Seneca, and Epictetus.
        You provide insightful and practical advice based on Stoic philosophy.
        """,
        role=MessageRole.SYSTEM),
    ChatMessage(
        content="""
        We have provided context information below.
        {context_str}
        ---------------------
        Given this information, please answer the question: {query}
        ---------------------
        If the question is not from the provided context, say `I don't know. Not enough information received.`
        """,
        role=MessageRole.USER,
    ),
]

In [ ]:
def pipeline(query):
    # R - Retriver
    relevant_documents = search(query)
    context = [doc.payload['context'] for doc in relevant_documents.points]
    context = "\n".join(context)

    # A - Augment
    chat_template = ChatPromptTemplate(message_templates=message_templates)

    # G - Generate
    response = llm.complete(
        chat_template.format(
            context_str=context,
            query=query)
    )
    return response

In [ ]:
print(pipeline("What does Marcus Aurelius say about finding inner peace in a chaotic world?"))

<think>
Okay, so I need to figure out what Marcus Aurelius says about finding inner peace in a chaotic world based on the provided context. Let me start by reading through the context carefully.

The context includes several passages from what seems to be letters or writings by Seneca and Marcus Aurelius. The user provided a mix of texts, some of which are repeated. I notice that the first part is from Seneca, discussing the importance of dealing with real problems rather than getting bogged down in theoretical debates. He talks about the futility of future wisdom and the importance of focusing on the present. He also mentions that future goods are not yet present, so they can't be helpful now.

Then, the latter part seems to be from Marcus Aurelius. He discusses how we often fear things more than they deserve and how we suffer more in imagination than in reality. He advises not to be unhappy before the crisis comes because the dangers we fear might never arrive. He also talks about th

In [ ]:
print(pipeline("Explain this Stoic teaching from Marcus Aurelius: 'If you are distressed by anything external, the pain is not due to the thing itself, but to your estimate of it; and this you have the power to revoke at any moment.'"))

<think>
Okay, so I need to explain this Stoic teaching from Marcus Aurelius: "If you are distressed by anything external, the pain is not due to the thing itself, but to your estimate of it; and this you have the power to revoke at any moment." 

First, I remember that Stoicism focuses a lot on controlling one's thoughts and emotions rather than external events. Marcus Aurelius was a Stoic philosopher, so his teachings would align with that. The quote is saying that when something external upsets us, it's not the thing itself causing the pain, but rather our judgment or perception of it. 

So, the key idea here is that our emotions and reactions are based on our interpretations. If I'm upset about something, it's because I'm assigning a certain meaning to it. For example, if it rains on my wedding day, the rain itself isn't causing my distress; it's my belief that the day should be sunny that's making me upset. 

The Stoics would argue that we have the power to change our judgments. In

In [1]:
print(pipeline("Guide me on how to calm oneself?"))

NameError: name 'pipeline' is not defined